# Chapter 3. word2vec

Ch.2 에서는 `통계 기반 기법`으로 단어의 분산 표현을 얻었습니다.  
이번 장에서는 `추론 기반 기법`을 살펴보겠습니다.

추론 과정에 신경망을 이용하는데, 여기서 `word2vec`을 사용합니다.  
#### 이번 장의 목표: '단순한' word2vec 구현하기

## 3.1 추론 기반 기법과 신경망  


### 3.1.1. 통계 기반 기법의 문제점

통계 기반 기법은 **대규모 말뭉치**를 다룰 때 문제가 발생  
통계 기반 기법은 말뭉치 전체의 통계를 이용해 단 1회의 처리만에 단어의 분산 표현을 얻습니다.  
추론 기반 기법에서는, 신경망을 이용하는 경우는 **미니배치로 학습**하는 것이 일반적입니다.  

<img src='./master/images/fig 3-1.png' />

### 3.1.2 추론 기반 기법 개요

<p align="center"><img src="./master/images/fig 3-2.png" width=500 />

위 그림처럼 추론 문제를 풀고 학습하는 것이 `추론 기반 기법`이 다루는 문제

<p align="center"><img src="./master/images/fig 3-3.png" width=500>

### 3.1.3 신경망에서의 단어 처리

1. 단어를 '고정 길이의 벡터'로 변환 $\rightarrow$ `원핫 벡터` 로 변환  
2. 신경망을 구성하는 '계층'들은 벡터를 처리할 수 있다. $\rightarrow$ 단어를 신경망으로 처리할 수 있게됨  

<p align="center"><img src="./master/images/fig 3-7.png" width=500 />

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]]) # 입력
W = np.random.randn(7, 3) # 가중치
h = np.matmul(c, W)
print(h)

[[ 0.65261928 -0.40796078  0.88458879]]


In [3]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[1.14087741 1.78421055 0.12346721]]


## 3.2 단순한 word2vec

<img src="./master/images/fig 3-3.png">

위 그림의 `모델`을 신경망으로 구축해보자!

이번 절에서는 모델로 `CBOW(Continuous bag-of-words)` 를 사용

### 3.2.1 CBOW 모델의 추론 처리

<p align="center"><img src="./master/images/fig 3-9.png" width=500 />

- `CBOW`의 입력 : '맥락'을 원핫 벡터로 변환한 벡터
- 위에서 입력층이 2개인 이유는 `window_size` 가 1이기 때문에 (맥락으로 고려할 단어가 2개)
- `CBOW`의 출력 : 각 단어의 '점수'. 값이 높을수록 대응 단어의 출현 확률이 높아진다. 
- $W_{in}$ : 해당 단어의 분산 표현이 담겨있음

In [4]:
# CBOW 모델의 추론 처리 구현
import sys 
sys.path.append('..')
import numpy as np
from common.layers import MatMul

# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기화
W_in = np.random.rand(7, 3)
W_out = np.random.rand(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파 
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[0.64326783 0.83863979 0.44822332 0.58972759 0.79366845 0.40870303
  0.64941756]]


### 3.2.2 CBOW 모델의 학습

<img src="./master/images/fig 3-12.png" width=400 align='left'/><img src="./master/images/fig 3-13.png" width=450 align='right'/>

- 추론 처리를 수행하는 `CBOW` 모델에 `Softmax` 와 `Cross Entropy Error` 계층을 추가

### 3.2.3 word2vec의 가중치와 분산 표현

- 단어의 분산 표현으로 $W_{in}$ 만을 사용

## 3.3 학습 데이터 준비

### 3.3.1 맥락과 타깃

- `타깃` : 맥락에 둘러싸인 중앙의 단어

#### 1. 말뭉치 텍스트를 단어 ID로 변환

In [5]:
import sys 
sys.path.append('..')
from common.util import preprocess

text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print()
print(id_to_word)

[0 1 2 3 4 1 5 6]

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


#### 2. corpus로부터 `맥락`과 `타깃`을 생성

In [7]:
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    contexts = []
    
    for idx in range(window_size, len(corpus)-window_size):
        cs = [] 
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)
        
    return np.array(contexts), np.array(target)

In [8]:
contexts, target = create_contexts_target(corpus, window_size=1)

print(contexts)
print()
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]

[1 2 3 4 1 5]


### 3.3.2 원핫 표현으로 변환

#### 3. 단어 ID를 원핫 벡터로 변환

In [9]:
def convert_one_hot(corpus, vocab_size):
    '''원핫 표현으로 변환
    
    :param corpus: 단어 ID 목록(1차원 또는 2차원 넘파이 배열)
    :param vocab_size: 어휘 수
    :return: 원핫 표현 (2차원 또는 3차원 넘파이 배열)
    '''
    
    N = corpus.shape[0]
    
    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1
            
    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1
                
    return one_hot

In [10]:
import sys
sys.path.append('..')
from common.util import preprocess, create_contexts_target, convert_one_hot

text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [11]:
print(target)

[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]


## 3.4 CBOW 모델 구현

<p align="center"><img src="./master/images/fig 3-19.png" />